In [1]:
!pip install opencv-python dlib imutils


DEPRECATION: Loading egg at c:\python\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from imutils import face_utils
import cv2

In [2]:
import numpy as np

In [3]:
# Dlib for deep learning based Modules and face landmark detection
import dlib

In [4]:
cap = cv2.VideoCapture(0)
# if cap.isOpened():
#     print(f"Camera founded")
#     cap.release()
# else:
#     print(f"No camera not founded ")
    

In [5]:
if not cap.isOpened():
    print("Error: Could not open video source.")
    exit()

In [6]:
#Initializing the face detector and landmark detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [7]:
sleep = 0
drowsy = 0
active = 0
status=""
color=(0,0,0)

In [9]:
 # Calculate the Euclidean distance between two points
def compute(ptA,ptB):
	dist = np.linalg.norm(ptA - ptB)
	return dist

In [10]:
def blinked(a, b, c, d, e, f):
    # Sum of the distances between upper eyelid points
    up = compute(b, d) + compute(c, e)
    # Distance between lower eyelid points
    down = compute(a, f)
    # Compute the eye aspect ratio (EAR)
    ratio = up / (2.0 * down)

    # Checking if it is blinked
    if ratio > 0.25:
        return 2  # Eyes are open
    elif 0.21 < ratio <= 0.25:
        return 1  # Eyes are partially open
    else:
        return 0  # Eyes are closed


In [11]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Error: Could not read frame.")
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform face detection
    faces = detector(gray)

    # Copy the frame to draw landmarks and rectangles
    face_frame = frame.copy()

    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Predict landmarks
        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        # Detect blinks
        left_blink = blinked(landmarks[36], landmarks[37], 
                             landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42], landmarks[43], 
                              landmarks[44], landmarks[47], landmarks[46], landmarks[45])
        
        # Judge what to do for the eye blinks
        if left_blink == 0 or right_blink == 0:
            sleep += 1
            drowsy = 0
            active = 0
            if sleep > 6:
                status = "SLEEPING !!!"
                color = (255, 0, 0)
        elif left_blink == 1 or right_blink == 1:
            sleep = 0
            active = 0
            drowsy += 1
            if drowsy > 2:
                status = "Drowsy !"
                color = (0, 0, 255)
        else:
            drowsy = 0
            sleep = 0
            active += 1
            if active > 2:
                status = "Active :)"
                color = (0, 255, 0)
        
        cv2.putText(frame, status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

        for (x, y) in landmarks:
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

    # Display the resulting frame
    cv2.imshow("Frame", frame)
    cv2.imshow("Result of detector", face_frame)
    
    key = cv2.waitKey(1)
    if key == 27:  # Exit on ESC key
        break

# Release the capture and close any OpenCV windows
cap.release()
cv2.destroyAllWindows()

Error: Could not read frame.
